# Cassava Leaf Disease Classification

## Prepare Data

### Import libaries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import json
import seaborn as sns
import os
import tensorflow as tf

from PIL import Image
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization, LeakyReLU, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import models
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.layers.experimental.preprocessing import RandomRotation, RandomFlip, RandomZoom, Rescaling

### Set seed to default output

In [ ]:
seed = 2020
np.random.seed(seed)
tf.random.set_seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

### Review dataframe

In [ ]:
path = '../input/cassava-leaf-disease-classification'
df = pd.read_csv(path + "/train.csv", sep = ',')
print ('Dataframe Shape: ', df.shape)
df.head()

In [ ]:
fig = plt.figure(figsize=(8,6))
sns.set(font_scale = 1.0)
label = sns.countplot(x = "label", data = df, order = df.label.value_counts().index)

plt.xlabel("Label", fontsize = 12)
plt.ylabel("Count", fontsize = 12)
plt.show()

In [ ]:
with open(path + "/label_num_to_disease_map.json") as f:
        label_names = json.loads(f.read())
        label_names = {int(i): j for i,j in label_names.items()}

label_names

In [ ]:
df.label = df.label.astype('string')
df.info()

## Load Data

### Pictures of different diseases

In [ ]:
df_images_path = path + "/train_images/"

In [ ]:
fig = plt.figure(figsize = (8,8))
npicture = 9

count = 1
images_list = df[df.label == str(list(label_names.keys())[list(label_names.values()).index("Cassava Bacterial Blight (CBB)")])]["image_id"].sample(frac = 1)[:npicture].to_list()
for i, img in enumerate(images_list):
    sample = os.path.join(df_images_path, img)
    sample_img = Image.open(sample)
    ax = fig.add_subplot(int(npicture / 3), 3, count, xticks = [], yticks = [])
    plt.imshow(sample_img)
    count +=1

fig.suptitle("Cassava Bacterial Blight (CBB)")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize = (8,8))
npicture = 9

count = 1
images_list = df[df.label == str(list(label_names.keys())[list(label_names.values()).index("Cassava Brown Streak Disease (CBSD)")])]["image_id"].sample(frac = 1)[:npicture].to_list()
for i, img in enumerate(images_list):
    sample = os.path.join(df_images_path, img)
    sample_img = Image.open(sample)
    ax = fig.add_subplot(int(npicture / 2), 3, count, xticks = [], yticks = [])
    plt.imshow(sample_img)
    count +=1

fig.suptitle("Cassava Brown Streak Disease (CBSD)")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize = (8,8))
npicture = 9

count = 1
images_list = df[df.label == str(list(label_names.keys())[list(label_names.values()).index("Cassava Green Mottle (CGM)")])]["image_id"].sample(frac = 1)[:npicture].to_list()
for i, img in enumerate(images_list):
    sample = os.path.join(df_images_path, img)
    sample_img = Image.open(sample)
    ax = fig.add_subplot(int(npicture / 2), 3, count, xticks = [], yticks = [])
    plt.imshow(sample_img)
    count +=1

fig.suptitle("Cassava Green Mottle (CGM)")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize = (8,8))
npicture = 9

count = 1
images_list = df[df.label == str(list(label_names.keys())[list(label_names.values()).index("Cassava Mosaic Disease (CMD)")])]["image_id"].sample(frac = 1)[:npicture].to_list()
for i, img in enumerate(images_list):
    sample = os.path.join(df_images_path, img)
    sample_img = Image.open(sample)
    ax = fig.add_subplot(int(npicture / 2), 3, count, xticks = [], yticks = [])
    plt.imshow(sample_img)
    count +=1

fig.suptitle("Cassava Mosaic Disease (CMD)")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize = (8,8))
npicture = 9

count = 1
images_list = df[df.label == str(list(label_names.keys())[list(label_names.values()).index("Healthy")])]["image_id"].sample(frac = 1)[:npicture].to_list()
for i, img in enumerate(images_list):
    sample = os.path.join(df_images_path, img)
    sample_img = Image.open(sample)
    ax = fig.add_subplot(int(npicture / 2), 3, count, xticks = [], yticks = [])
    plt.imshow(sample_img)
    count +=1

fig.suptitle("Healthy")
plt.tight_layout()
plt.show()

### Check image size if all are same or not

In [ ]:
im_name_lists = df['image_id'].tolist()
im_shape_x_lists = []
im_shape_y_lists = []
for i, img in enumerate(im_name_lists):
    sample = os.path.join(df_images_path, img) 
    sample_img = Image.open(sample)
    w, h = sample_img.size
    im_shape_x_lists.append(w)
    im_shape_y_lists.append(h)

print ('Check: ', len(im_shape_x_lists), len(im_shape_y_lists))

In [ ]:
fig = plt.figure(figsize=(6, 6))

fig.add_subplot(121)
plt.hist(im_shape_x_lists)
fig.add_subplot(122)
plt.hist(im_shape_y_lists)
plt.tight_layout()

print(set(im_shape_x_lists), set(im_shape_y_lists))

### Resize image 

In [ ]:
target_size = (300, 300)
input_shape = (300, 300, 3)
batch_size = 64
seed = 10
epochs = 5

## InceptionResNetV2

### Create the Batch Generators

In [ ]:
data_generator = ImageDataGenerator(validation_split = 0.2)

train_data = data_generator.flow_from_dataframe(df, 
                                                directory = df_images_path, 
                                                x_col = "image_id",
                                                y_col = "label",
                                                target_size = target_size,
                                                batch_size = batch_size,
                                                shuffle = True, 
                                                seed = seed,
                                                class_mode = "sparse",
                                                subset = "training")

val_data = data_generator.flow_from_dataframe(df, 
                                                directory = df_images_path, 
                                                x_col = "image_id",
                                                y_col = "label",
                                                target_size = target_size,
                                                batch_size = batch_size,
                                                shuffle = True, 
                                                seed = seed,
                                                class_mode = "sparse",
                                                subset = "validation")

In [ ]:
fig = plt.figure(figsize=(15, 10))
npics= 16
count = 1
for i in range(npics):
    x,y = val_data.next()
    image = x[0].astype('uint8')

    label = y[0]  
    int_label = int(label)  
    ax = fig.add_subplot(int(npics/4) , 4, count, xticks=[],yticks=[])
    ax.set_title(label_names[int_label], fontsize = 10)  
    plt.imshow(image)
    count = count + 1  

plt.tight_layout()
plt.show()

### Build Model With InceptionResNetV2

In [ ]:
class customCallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = None):
        self.epoch = epoch + 1
        if self.epoch % 2 == 0:
            print('epoch num {}, train loss: {}, validation loss: {}'.format(epoch, logs['loss'], logs['val_loss']))

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', patience = 2, verbose=1)
mcp_save = ModelCheckpoint(filepath = "best_model_weights.h5",
                           save_best_only = True, save_weights_only = True, monitor = 'val_loss')
es = EarlyStopping(monitor = "val_loss", patience = 10)

In [ ]:
inception_resnet_v2 = InceptionResNetV2(
    include_top = False,
    weights = "../input/inceptionresnetv2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5",
    input_shape = input_shape,)

def build_model():
    inputs = Input(input_shape)
    
    x = preprocess_input(inputs)
    x = Rescaling(1./255)(x)
    
    x = RandomFlip()(x)
    x = RandomRotation(factor=0.3)(x)
    
    x = BatchNormalization()(x)
    x = inception_resnet_v2(x)

    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(256, (1, 1), activation=LeakyReLU())(x)
    x = BatchNormalization()(x)
    
    x = Flatten()(x)
    x = Dropout(0.75)(x)

    x = Dense(256, activation=LeakyReLU())(x)
    x = Dropout(0.80)(x)
    x = BatchNormalization()(x)
    
    outputs = Dense(5, activation="softmax")(x)
    
    model = Model(inputs, outputs)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                  loss="sparse_categorical_crossentropy", 
                  metrics=["accuracy"])
    
    return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
history = model.fit(train_data, 
                    validation_data = val_data, 
                    epochs = epochs, 
                    callbacks = [mcp_save, es, reduce_lr])

### Check if the model is still fine after loading the trained weights

In [ ]:
model.load_weights("best_model_weights.h5")
model.evaluate(val_data)

### Training example image 

In [ ]:
ex_im = pd.read_csv("../input/cassava-leaf-disease-classification/sample_submission.csv")
ex_im.head()

In [ ]:
fig = plt.figure(figsize = (8,8))

sample = os.path.join('../input/cassava-leaf-disease-classification/test_images/2216849948.jpg')
sample_img = Image.open(sample)
ax = fig.add_subplot(xticks = [], yticks = [])
plt.imshow(sample_img)

fig.suptitle("Picture to Test")
plt.tight_layout()
plt.show()

In [ ]:
test_images = os.listdir('../input/cassava-leaf-disease-classification/test_images/')
preds = []

for i in test_images:
    image = Image.open(f'../input/cassava-leaf-disease-classification/test_images/{i}')
    image = image.resize(target_size)
    image = np.expand_dims(image, axis = 0)
    preds.append(np.argmax(model.predict(image)))

In [ ]:
df_ans = pd.DataFrame({'image_id': test_images, 'label': preds})
df_ans.head()

In [ ]:
df_ans.to_csv("submission.csv", index=None)

### Training and Validation Curves

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

fig = plt.figure(figsize=(15, 5))
fig.add_subplot(121)

plt.plot(epochs, acc, linestyle = '--', label = "Training acc")
plt.plot(epochs, val_acc, linestyle = '-.', label = "Validation acc")
plt.title("Training and validation acc")
plt.legend()

fig.add_subplot(122)
plt.plot(epochs, loss, linestyle = '--', label = "Training loss", alpha = 0.8)
plt.plot(epochs, val_loss, linestyle = '-.', label = "Validation loss", alpha = 0.6)
plt.title("Training and validation loss")
plt.legend()

plt.show()

In [ ]:
class_types = list(label_names.values())

def conf_matrix(test_lab, predictions): 
    cm = confusion_matrix(test_lab, np.argmax(np.round(predictions), axis = 1))
    print("Classification Report:\n")
    
    cr = classification_report(test_lab,
                                np.argmax(np.round(predictions), axis = 1), 
                                target_names = [class_types[i] for i in range(len(class_types))])
    print(cr)
    
    plt.figure(figsize = (8,8))
    sns_hmp = sns.heatmap(cm, annot = True, xticklabels = [class_types[i] for i in range(len(class_types))], 
                yticklabels = [class_types[i] for i in range(len(class_types))], fmt = "d")
    fig = sns_hmp.get_figure()

### Plot the Confusion Matrix (Validation Data)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

npics = 300
val_pre = []
val_img = []
val_label = []
for _ in range(npics):
    x,y = val_data.next()
    image = x[0].astype('uint8')
    label = y[0]
    val_label.append(label)
    image = np.expand_dims(image, axis = 0)
    val_img.append(image)

In [ ]:
val_img_arr = np.array(val_img)
val_label_arr = np.array(val_label)
val_img_arr = np.reshape(val_img_arr, (300, 300, 300, 3))

In [ ]:
pred_class_InceptResV2 = model.predict(val_img_arr)

In [ ]:
conf_matrix(np.int32(val_label_arr), pred_class_InceptResV2)

## Xception

### Create the Batch Generators

In [ ]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=0.5,
                                                                 horizontal_flip=True,
                                                                 vertical_flip=True,
                                                                 brightness_range=[0.1,0.3],
                                                                 zoom_range=[0.1,0.5],
                                                                 fill_mode='nearest',
                                                                 preprocessing_function=tf.keras.applications.xception.preprocess_input,
                                                                 validation_split=0.2,
                                                                  rescale = 1./255,
                                                                  dtype='float32')
train_generator = image_generator.flow_from_dataframe(df, 
                                                directory = df_images_path, 
                                                x_col = "image_id",
                                                y_col = "label",
                                                target_size = target_size,
                                                batch_size = batch_size,
                                                shuffle = True, 
                                                seed = seed,
                                                class_mode = "categorical",
                                                subset = "training")
val_generator = image_generator.flow_from_dataframe(df, 
                                                directory = df_images_path, 
                                                x_col = "image_id",
                                                y_col = "label",
                                                target_size = target_size,
                                                batch_size = batch_size,
                                                shuffle = True, 
                                                seed = seed,
                                                class_mode = "categorical",
                                                subset = "validation")

### Build Model With Xception

In [ ]:
def cnn_model_pretrain(xception=True,shape=(300,300,3),filters=8,kernel=4,stride=1,pad='same',unit=1024,activation=tf.nn.relu,
              regularizer=tf.keras.regularizers.L1L2(l1=0.01,l2=0.01)):
    
    '''function to define the cnn model architecture with or without transfer learning'''
    
    input_img = tf.keras.Input(shape=shape)
    
    base_model = tf.keras.applications.xception.Xception(include_top=False,input_shape=shape,weights='imagenet')
    for layer in base_model.layers:
        layer.trainable = False
    
    if xception:
        pretrain = base_model(input_img)
    else:
        pretrain =  input_img
        
    '''One layer of Conv2D, Activation and Pooling layer'''
    Z1 = tf.keras.layers.Conv2D(filters=filters,kernel_size=(kernel,kernel),strides=stride,padding=pad)(pretrain)
    A1 = tf.keras.layers.Activation(activation)(Z1)
    P1 = tf.keras.layers.MaxPool2D(pool_size=(filters,filters),strides=stride*8,padding=pad)(A1)
    
    '''Second layer of Conv2D, Activation and Pooling layer'''
    Z2 = tf.keras.layers.Conv2D(filters=filters*2,kernel_size=(int(kernel/2),int(kernel/2)),strides=stride,
                               padding=pad) (P1)
    A2 = tf.keras.layers.Activation(activation)(Z2)
    P2 = tf.keras.layers.MaxPool2D(pool_size=int(filters/2),strides=stride*4,padding=pad)(A2)
    
    '''Flatten the output to feed into the dense layer'''
    F = tf.keras.layers.Flatten()(P2)
    
    '''Dense Layers'''
    D1 = tf.keras.layers.Dense(units=unit,activation=activation)(F)
    D2 = tf.keras.layers.Dense(units=unit/2,activation=activation,kernel_regularizer=regularizer)(D1)
    D3 = tf.keras.layers.Dense(units=unit/4,activation=activation,kernel_regularizer=regularizer)(D2)
    
    '''Output layer'''
    outputs = tf.keras.layers.Dense(units=5,activation=tf.nn.softmax)(D3)
    
    '''instantiate the model'''
    model = tf.keras.Model(inputs=input_img,outputs=outputs)
    return model  

In [ ]:
regularizer = tf.keras.regularizers.L2(l2=0.005)
shape = (300,300,3)

conv_model = cnn_model_pretrain(xception=True,shape=shape,filters=8,kernel=4,stride=1,pad='same',unit=256,activation=tf.nn.relu,
                      regularizer=regularizer)

conv_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
conv_model.summary()

In [ ]:
history = conv_model.fit(train_generator,epochs = 5,validation_data = val_generator,batch_size = batch_size)

### Training and Validation Curves

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

fig = plt.figure(figsize=(15, 5))
fig.add_subplot(121)

plt.plot(epochs, acc, linestyle = '--', label = "Training acc")
plt.plot(epochs, val_acc, linestyle = '-.', label = "Validation acc")
plt.title("Training and validation acc")
plt.legend()

fig.add_subplot(122)
plt.plot(epochs, loss, linestyle = '--', label = "Training loss", alpha = 0.8)
plt.plot(epochs, val_loss, linestyle = '-.', label = "Validation loss", alpha = 0.6)
plt.title("Training and validation loss")
plt.legend()

plt.show()